In [ ]:
#!pip install gdown
import os
import gdown
import numpy as np
import pandas as pd
from zipfile import ZipFile
from IPython.display import clear_output as cls, Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
cls()

In [ ]:
url = 'https://drive.google.com/uc?id=1ZbpfUKMYbrzIdhZ5UByibCmJ8JaSP59R'
output = 'data.zip'
gdown.download(url, output, quiet=False)
with ZipFile(file="/content/data.zip", mode='r') as Zfile:
    Zfile.extractall(path="/content/data")
cls()

In [ ]:
dataset = pd.read_csv('/content/data/TMDB_movie_dataset_v11.csv')

In [ ]:
dataset.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185194 entries, 0 to 1185193
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1185194 non-null  int64  
 1   title                 1185181 non-null  object 
 2   vote_average          1185194 non-null  float64
 3   vote_count            1185194 non-null  int64  
 4   status                1185194 non-null  object 
 5   release_date          977506 non-null   object 
 6   revenue               1185194 non-null  int64  
 7   runtime               1185194 non-null  int64  
 8   adult                 1185194 non-null  bool   
 9   backdrop_path         311072 non-null   object 
 10  budget                1185194 non-null  int64  
 11  homepage              124732 non-null   object 
 12  imdb_id               614389 non-null   object 
 13  original_language     1185194 non-null  object 
 14  original_title        1185181 non-

In [ ]:
dataset.shape

(1185194, 24)

In [ ]:
dataset['vote_count'].nunique()

3598

In [ ]:
dataset.isnull().sum()

,0
id,0
title,13
vote_average,0
vote_count,0
status,0
release_date,207688
revenue,0
runtime,0
adult,0
backdrop_path,874122


In [ ]:
dataset.duplicated('title').sum()

170398

In [ ]:
dataset.drop_duplicates('title', inplace=True)

In [ ]:
dataset.dropna(subset=['title', 'genres', 'overview', 'keywords'], inplace=True)
dataset.fillna({
    'release_date': 'Unknown',
    'backdrop_path': '-',
    'homepage': '-',
    'imdb_id': '-',
    'original_title': 'Unknown',
    'poster_path': '-',
    'tagline': '-',
    'production_companies': 'Unknown',
    'production_countries': 'Unknown',
    'spoken_languages': 'Unknown',
}, inplace=True)



In [ ]:
m = dataset['vote_count'].quantile(0.9)
C = dataset['vote_average'].mean()
dataset['weighted_rating'] = (dataset['vote_count']/(dataset['vote_count']+m) * dataset['vote_average']) + (m/(m+dataset['vote_count']) * C)

In [ ]:
dataset.drop(columns=['vote_average','vote_count'], inplace=True)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195933 entries, 0 to 1185183
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    195933 non-null  int64  
 1   title                 195933 non-null  object 
 2   status                195933 non-null  object 
 3   release_date          195933 non-null  object 
 4   revenue               195933 non-null  int64  
 5   runtime               195933 non-null  int64  
 6   adult                 195933 non-null  bool   
 7   backdrop_path         195933 non-null  object 
 8   budget                195933 non-null  int64  
 9   homepage              195933 non-null  object 
 10  imdb_id               195933 non-null  object 
 11  original_language     195933 non-null  object 
 12  original_title        195933 non-null  object 
 13  overview              195933 non-null  object 
 14  popularity            195933 non-null  float64
 15  post

In [ ]:
dataset['title'].iloc[2]

'The Dark Knight'

In [ ]:
dataset.shape

(195933, 23)

In [ ]:
dataset['combined_features'] = (
    dataset['genres'] + ' ' +
    dataset['keywords'] + ' ' +
    dataset['overview'] )

In [ ]:
dataset['combined_features'].iloc[1]

'Adventure, Drama, Science Fiction rescue, future, spacecraft, race against time, artificial intelligence (a.i.), nasa, time warp, dystopia, expedition, space travel, wormhole, famine, black hole, quantum mechanics, family relationships, space, robot, astronaut, scientist, single father, farmer, space station, curious, space adventure, time paradox, thoughtful, time-manipulation, father daughter relationship, 2060s, cornfield, time manipulation, complicated The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.'

In [ ]:
nltk.download(['stopwords', 'wordnet', 'omw-1.4'])
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  tokens = text.split()
  cleaned_tokens = [ lemmatizer.lemmatize(word)
                      for word in tokens
                    if word not in stop_words and len(word) > 2
                    ]

  return ' '.join(cleaned_tokens)
def clean_title(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(cleaned_tokens)


In [ ]:
dataset['combined_features'] = dataset['combined_features'].apply(clean_text)

In [ ]:
dataset['clean_title'] = dataset['title'].apply(clean_title)

In [ ]:
dataset['clean_title'].iloc[1]

'interstellar'

In [ ]:
dataset['features'] = (
    dataset['clean_title'] + ' ' +
    dataset['combined_features'])

In [ ]:
dataset['features'].iloc[0]

'inception action science fiction adventure rescue mission dream airplane paris france virtual reality kidnapping philosophy spy allegory manipulation car crash heist memory architecture los angeles california dream world subconscious cobb skilled thief commits corporate espionage infiltrating subconscious target offered chance regain old life payment task considered impossible inception implantation another person idea target subconscious'

In [ ]:
dataset

,id,title,status,release_date,revenue,runtime,adult,backdrop_path,budget,homepage,...,tagline,genres,production_companies,production_countries,spoken_languages,keywords,weighted_rating,combined_features,clean_title,features
0,27205,Inception,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,160000000,https://www.warnerbros.com/movies/inception,...,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...",8.355628,action science fiction adventure rescue missio...,inception,inception action science fiction adventure res...
1,157336,Interstellar,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,165000000,http://www.interstellarmovie.net/,...,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,...",8.408029,adventure drama science fiction rescue future ...,interstellar,interstellar adventure drama science fiction r...
2,155,The Dark Knight,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,185000000,https://www.warnerbros.com/movies/dark-knight/,...,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",8.502257,drama action crime thriller joker sadism chaos...,the dark knight,the dark knight drama action crime thriller jo...
3,19995,Avatar,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,237000000,https://www.avatar.com/movies/avatar,...,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...",7.565037,action adventure fantasy science fiction futur...,avatar,avatar action adventure fantasy science fictio...
4,24428,The Avengers,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,220000000,https://www.marvel.com/movies/the-avengers,...,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",7.701556,science fiction action adventure new york city...,the avenger,the avenger science fiction action adventure n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185057,756960,The Girl Who Comes On Thursdays,Released,2020-08-21,0,19,False,-,0,-,...,-,Drama,Islandia Producciones,Spain,Galician,"loneliness, elderly lady",3.912624,drama loneliness elderly lady world already li...,the girl who come on thursday,the girl who come on thursday drama loneliness...
1185125,756689,Hot Chocolate with Marshmallows,Released,2019-10-12,0,11,False,-,1000,-,...,-,Drama,Unknown,United States of America,English,"grief, grieving, grieving parents",3.912624,drama grief grieving grieving parent uppermidd...,hot chocolate with marshmallow,hot chocolate with marshmallow drama grief gri...
1185128,756693,Bunuwirah,Released,2020-10-15,0,95,False,/yrssvVTehgLqL0i39xDnTVjA8XH.jpg,350000,https://bunuwirah.com/,...,Do not follow the light,"Horror, Thriller",Abdullah Abuljadail Films,Saudi Arabia,Arabic,"friendship, supernatural, demon, desert, found...",3.912624,horror thriller friendship supernatural demon ...,bunuwirah,bunuwirah horror thriller friendship supernatu...
1185137,756702,Wesens,Released,2020-10-23,0,90,False,/bIeEleP12kx7BQJ2w2M2fbIaoX8.jpg,0,http://www.wesens.co.za/,...,-,"Mystery, Science Fiction",Ambassadeur,South Africa,Afrikaans,"found footag

In [ ]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 3))
tfidf_matrix = tfidf.fit_transform(dataset['features'])

In [ ]:
import pickle
import scipy.sparse

with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

scipy.sparse.save_npz("tfidf_matrix.npz", tfidf_matrix)


In [ ]:
print("inception" in tfidf.get_feature_names_out())


True


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(user_input, top_n=10):
    user_input = user_input.strip()

    if not user_input:
        print("Masukkan tidak boleh kosong. Silakan coba lagi.")
        return None

    if user_input.lower() in dataset['title'].str.lower().values:
        idx = dataset[dataset['title'].str.lower() == user_input.lower()].index[0]
        sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)
    else:  
        input_cleaned = clean_text(user_input)
        input_vector = tfidf.transform([input_cleaned])
        sim_scores = cosine_similarity(input_vector, tfidf_matrix)

    if np.max(sim_scores) == 0:
        print("Maaf, tidak ditemukan film yang cocok.")
        return None

    sim_scores = sim_scores.flatten()
    top_indices = sim_scores.argsort()[::-1][:top_n]

    recommendations = dataset.iloc[top_indices][['title']].copy()
    recommendations['similarity_score'] = sim_scores[top_indices]

    return recommendations

if __name__ == "__main__":
    while True:
        user_input = input("\nMasukkan judul film atau keywords (ketik 'exit' untuk keluar): ")
        if user_input.lower() == 'exit':
            break

        results = get_recommendations(user_input)
        if results is not None:
            print("\nRekomendasi:")
            print(results.to_string(index=False))



Masukkan judul film atau keywords (ketik 'exit' untuk keluar): dream cinema of the subconscious

Rekomendasi:
                                        title  similarity_score
           Dreams: Cinema of the Subconscious          0.205753
                               Cinema Company          0.193405
                            Slumber is Golden          0.143930
                                    Inception          0.127343
            Return to Manila: Filipino Cinema          0.118426
                              The Last Cinema          0.115470
                   A Trip to the Subconscious          0.113984
                    The Holly Kane Experiment          0.102242
Secrets of the Unknown: Dreams and Nightmares          0.102142
                                  Sotterranea          0.101289

Masukkan judul film atau keywords (ketik 'exit' untuk keluar): exit


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()

In [ ]:
count_matrix = CV.fit_transform(dataset['features'])


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(user_input, top_n=10):
    user_input = user_input.strip()

    if not user_input:
        print("Masukkan tidak boleh kosong. Silakan coba lagi.")
        return None

    if user_input.lower() in dataset['title'].str.lower().values:
        idx = dataset[dataset['title'].str.lower() == user_input.lower()].index[0]
        sim_scores = cosine_similarity(count_matrix[idx], count_matrix)
    else:  
        input_cleaned = clean_text(user_input)
        input_vector = tfidf.transform([input_cleaned])
        sim_scores = cosine_similarity(input_vector, count_matrix).flatten()

    if np.max(sim_scores) == 0:
        print("Maaf, tidak ditemukan film yang cocok.")
        return None

    sim_scores = sim_scores.flatten()
    top_indices = sim_scores.argsort()[::-1][:top_n]

    recommendations = dataset.iloc[top_indices][['title']].copy()
    recommendations['similarity_score'] = sim_scores[top_indices]

    return recommendations

if __name__ == "__main__":
    while True:
        user_input = input("\nMasukkan judul film atau keywords (ketik 'exit' untuk keluar): ")
        if user_input.lower() == 'exit':
            break

        results = get_recommendations(user_input)
        if results is not None:
            print("\nRekomendasi:")
            print(results.to_string(index=False))



Masukkan judul film atau keywords (ketik 'exit' untuk keluar): inception

Rekomendasi:
                                        title  similarity_score
                                    Inception          1.000000
           Dreams: Cinema of the Subconscious          0.275241
                                Sommer Dreams          0.268608
                   A Trip to the Subconscious          0.266698
                            Slumber is Golden          0.262432
                               Olivia's Dream          0.246183
Secrets of the Unknown: Dreams and Nightmares          0.235935
                     The Matrix Resurrections          0.227921
                                     Calliope          0.226134
                            A Thing of Dreams          0.225668

Masukkan judul film atau keywords (ketik 'exit' untuk keluar): exit


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195933 entries, 0 to 1185183
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    195933 non-null  int64  
 1   title                 195933 non-null  object 
 2   status                195933 non-null  object 
 3   release_date          195933 non-null  object 
 4   revenue               195933 non-null  int64  
 5   runtime               195933 non-null  int64  
 6   adult                 195933 non-null  bool   
 7   backdrop_path         195933 non-null  object 
 8   budget                195933 non-null  int64  
 9   homepage              195933 non-null  object 
 10  imdb_id               195933 non-null  object 
 11  original_language     195933 non-null  object 
 12  original_title        195933 non-null  object 
 13  overview              195933 non-null  object 
 14  popularity            195933 non-null  float64
 15  post

In [ ]:
dataset.to_csv("movie_data.csv")